In [16]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()
try:
    from . import scrapper
except:
    import scrapper


In [17]:
list_pick_members1 = ["1.1", "1.2", "1.3", "1.4", "1.5", "1.6", "1.7", "1.8", "1.9", "1.10", "1.11", "1.12", "1.14", "1.15"]

list_pick_members2 = ["2.2", "2.3"]

pid=str(1410020101)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear="2022"
endMonth="12"
endYear="2022"
referencePeriods="20220101%2C20221001"

filter_names = ["Geography", "Type of Employee"]


In [18]:
combined_list_picker = [(x, y) for x in list_pick_members1 for y in list_pick_members2]

In [19]:
pid=str(1410020101)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear="2022"
endMonth="12"
endYear="2022"
referencePeriods="20220101%2C20221001"

filter_names = ["Geography", "Type of employee"]
    

In [20]:
table_name = "monthly_employment_by_industry"

# Connecting to Planet Scale
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + os.getenv("USERNAME") + ':' + os.getenv("PASSWORD") + '@' + os.getenv("HOST") + '/' + os.getenv("DATABASE") 

for x, y in combined_list_picker:
    url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=' + referencePeriods
    df = scrapper.simple_scrapper(url, filter_names)
    engine = create_engine(conn_string, connect_args=ssl_args)
    with engine.begin() as engine:
        df.to_sql(table_name, engine, if_exists='append', index=False)
        print("Inserted data from " + x + " and " + y + " into " + table_name + " successfully")

Inserted data from1.1 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.1 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.2 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.2 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.3 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.3 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.4 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.4 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.5 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.5 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.6 and 2.2 into monthly_employment_by_industry successfully
Inserted data from1.6 and 2.3 into monthly_employment_by_industry successfully
Inserted data from1.7 and 2.2 into monthly_employmen

In [21]:
import pandas as pd
read_engine = create_engine(conn_string, connect_args=ssl_args)

df = pd.read_sql_table(table_name, read_engine)
df.to_csv("./data/monthly_employment_by_industry.csv", index=False)
